In [6]:
import csv
import itertools
import pandas as pd
import numpy as np
from math import log, e
import random

In [5]:
# Add Methods notebook to this notebook
%run Methods.ipynb

Methods.ipynb:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  "execution_count": 77,


In [159]:
differenceNeighbourDist()

2.1469940658762883

In [160]:
distNeighbours()

50.44033027100041

# Greedy

In [7]:
# Iter Greedy algorithm

routingMap_nodesG = []
routingMap_distG = []

greedyIter(83, routingMap_distG, routingMap_nodesG)

In [ ]:
# Calculate entropy for each path

nodes_entropyG = []
dist_entropyG = []

for i in range(len(routingMap_nodesG)):
    nodes_entropyG.append(entropy(routingMap_nodesG[i]))
    dist_entropyG.append(entropy(routingMap_distG[i]))

In [ ]:
# Average entropy

sum_entG = 0
for i in range(6806):
    # sum_entG += dist_entropyG[i]
    sum_entG += nodes_entropyG[i]
    
avr_entG = sum_entG / 6806
avr_entG

# Dijkstra

In [ ]:
# Iter Dijsktra algorithm

routingMap_nodesD = []
routingMap_distD = []

dijkstraIter(83, routingMap_distD, routingMap_nodesD)

In [ ]:
# Calculate entropy for each path

nodes_entropyD = []
dist_entropyD = []

for i in range(len(routingMap_nodesD)):
    nodes_entropyD.append(entropy(routingMap_nodesD[i]))
    dist_entropyD.append(entropy(routingMap_distD[i]))

In [ ]:
# Average entropy

sum_entD = 0
for i in range(6806):
    # sum_entD += dist_entropyD[i]
    sum_entD += nodes_entropyD[i]
    
avr_entD = sum_entD / 6806
avr_entD

# Rand

In [ ]:
'''# Iter Rand algorithm
entropyRand = []

for i in range(100):
    print(i)
    tmpNode = []
    tmpDist = []
    randIter(83, tmpDist, tmpNode)
    if entropyRand == []:
        for j in tmpDist:
            entropyRand.append(entropy(j))
    else:
        for j in range(len(tmpDist)):
            entropyRand[j] += entropy(tmpDist[j])'''

# Random with boundaries

In [3]:
# Iter Rand with boundary algorithm
# Iterate the algorithm 100 times
'''entropyRand = []

for i in range(100):
    print(i)
    tmpNode = []
    tmpDist = []
    randBoundaryIter(83, tmpDist, tmpNode, 5)
    if entropyRand == []:
        for j in tmpDist:
            entropyRand.append(entropy(j))
    else:
        for j in range(len(tmpDist)):
            entropyRand[j] += entropy(tmpDist[j])'''

'entropyRand = []\n\nfor i in range(100):\n    print(i)\n    tmpNode = []\n    tmpDist = []\n    randBoundaryIter(83, tmpDist, tmpNode, 5)\n    if entropyRand == []:\n        for j in tmpDist:\n            entropyRand.append(entropy(j))\n    else:\n        for j in range(len(tmpDist)):\n            entropyRand[j] += entropy(tmpDist[j])'

In [2]:
# printOut('entDistBoundary5', entropyRand)
# printOut('entDistNewDivided', entropyRand)

 # Print out

In [ ]:
'''with open('alg2nodes.csv', 'w', newline='') as csv_1:
  csv_out = csv.writer(csv_1)
  csv_out.writerows([routing_map_nodes2[index]] for index in range(0, len(routing_map_nodes2)))
    
with open('alg2dist.csv', 'w', newline='') as csv_1:
  csv_out = csv.writer(csv_1)
  csv_out.writerows([routing_map_dist2[index]] for index in range(0, len(routing_map_dist2)))
    
with open('alg3nodes.csv', 'w', newline='') as csv_1:
  csv_out = csv.writer(csv_1)
  csv_out.writerows([routing_map_nodes3[index]] for index in range(0, len(routing_map_nodes3)))
    
with open('alg3dist.csv', 'w', newline='') as csv_1:
  csv_out = csv.writer(csv_1)
  csv_out.writerows([routing_map_dist3[index]] for index in range(0, len(routing_map_dist3)))
    
with open('ent2nodes.csv', 'w', newline='') as csv_1:
  csv_out = csv.writer(csv_1)
  csv_out.writerows([nodes_entropy2[index]] for index in range(0, len(nodes_entropy2)))
    
with open('ent2dist.csv', 'w', newline='') as csv_1:
  csv_out = csv.writer(csv_1)
  csv_out.writerows([dist_entropy2[index]] for index in range(0, len(dist_entropy2)))
    
with open('ent3nodes.csv', 'w', newline='') as csv_1:
  csv_out = csv.writer(csv_1)
  csv_out.writerows([nodes_entropy3[index]] for index in range(0, len(nodes_entropy3)))
    
with open('ent3dist.csv', 'w', newline='') as csv_1:
  csv_out = csv.writer(csv_1)
  csv_out.writerows([dist_entropy3[index]] for index in range(0, len(dist_entropy3)))'''